### Intalling libs

In [6]:
#  pip install pydotplus
# !pip install graphviz
# !pip install IPython
# !pip install dtreeviz
# !pip install -q dtreeviz
# !pip install xgbfir
# !pip install openpyxl
# openpyxl.__version__
# !pip list
# !pip install LightGBM
# !pip install 'dask>2023.3.2'
# !pip install 'distributed>2023.3.2'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: zict
    Found existing installation: zict 2.1.0
    Uninstalling zict-2.1.0:
      Successfully uninstalled zict-2.1.0
  Attempting uninstall: distributed
    Found existing installation: distributed 2022.7.0
    Uninstalling distributed-2022.7.0:
      Successfully uninstalled distributed-2022.7.0


### Column description

| Variável 	| Descrição 	|
|:-:	|:-	|
| PassangerID 	| ID de identificação do passageiro(a) 	|
| Survived 	| se o passageiro(a) sobreviveu (0 = não, 1 = sim) 	|
| Pclass 	| classe do passageiro:<br>     * **1 = primeira**,<br>     * **2 = segunda**,<br>     * **3 = terceira** 	|
| name 	| nome do passageiro(a) 	|
| sex 	| sexo do passageiro(a) 	|
| age 	| idade do passageiro(a) 	|
| Sibsp 	| número de irmão(ãs)/esposo(a) à bordo 	|
| Parch 	| número de pais/filhos(as) à bordo 	|
| Ticket 	| número da passagem 	|
| Fare 	| preço da passagem 	|
| Cabin 	| cabine 	|
| Embarked 	| local que o passageiro(a) embarcou:<br>     * **C = Cherboug**,<br>     * **Q = Queenstown**,<br>     * **S = Southamption** 	|
| WikiId 	| ID de identificação do passageiro(a) segundo Wikipedia 	|
| Name_wiki 	| nome do passageiro(a) 	|
| Age_wiki 	| idade do passageiro(a) 	|
| Hometown 	| cidade de nascimento do passageiro(a) 	|
| Boarded 	| cidade de embarque 	|
| Destination 	| destino da viagem 	|
| Lifeboat 	| identificação do bote salva-vidas 	|
| Body 	| número de identificação do corpo 	|


<font color='red'>**IMPORTANT**</font>

The new features (the ones after 'Embarked') are very similar to the original ones but they are more up-to-date and have much fewer missing values. Therefore, users can decide on the preferred features themselves.

### Importing Libs

In [7]:
# data visualization
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import (
    jointplot,
    pairplot,
    boxplot,
    heatmap
)

from yellowbrick.features import (
    Rank2D, 
    RadViz,
    FeatureImportances,
    ParallelCoordinates,
    JointPlotVisualizer,
)

import dtreeviz

import pydotplus

from io import(
    StringIO
)

from IPython.display import (
    Image
)

# data manipulation
import numpy as np
import pandas as pd
from pandas.plotting import(
    radviz
)
import janitor as jn

from ydata_profiling import ProfileReport

# missing values
import missingno as msno

from sklearn.impute import (
    SimpleImputer
)

# machine learning models
from sklearn import (
    ensemble,
    preprocessing,
    tree,
    impute,
    model_selection
)

from sklearn.utils import (
    resample
)

from sklearn.dummy import (
    DummyClassifier
)

from sklearn.model_selection import (
    train_test_split
)

from sklearn.experimental import (
    enable_iterative_imputer
)

from sklearn.linear_model import (
    LogisticRegression
)

from sklearn.naive_bayes import (
    GaussianNB
)

from sklearn.tree import (
    DecisionTreeClassifier,
    export_graphviz,
    plot_tree
)

from sklearn.neighbors import (
    KNeighborsClassifier
)

from sklearn.naive_bayes import (
    GaussianNB
)

from sklearn.svm import (
    SVC
)

from sklearn.ensemble import (
    RandomForestClassifier
)

from imblearn.over_sampling import (
    RandomOverSampler,
)

import rfpimp

import lightgbm as lgb

import xgboost as xgb

import xgbfir

# data model metrics
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_score,
    recall_score
)

from yellowbrick.classifier import (
    ConfusionMatrix
)

from yellowbrick.model_selection import (
    LearningCurve
)

# data prep-model
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    learning_curve
)

# model deploy
import pickle

### Reading the Titanic Dataset

In [ ]:
df = pd.read_csv("titanic_dataset.csv", index_col=0)
df.head(5)

### Deleting _Class_ feature at the end
We are deleting because is the same as _pclass_ (same result, same data)

In [ ]:
df = df.drop('Class', axis = 'columns')
df

### Converting DataFrame Column Names to Lowercase snakecase

In [ ]:
df.columns = (df.columns
                .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                .str.lower()
             )

df

### Preparing the dataset

In [ ]:
# dropping columns taht do not add value
df = df.drop(columns = ['name',
                        'name_wiki',
                        'wiki_id',
                        'hometown',
                        'destination',
                        'ticket',
                        'lifeboat',
                        'body',
                        'cabin',
                        'fare',
                        'age_wiki',
                        'age'])

# using get_dummies function to convert object to int
df = pd.get_dummies(df)

# dropping redundant features
df = df.drop(columns = ['sex_male'])

#remove rows with any values that are not finite (NaN or infite)
df = df[np.isfinite(df).all(1)]

# first, we need to create a series of the target feature
df = df.apply(lambda column: column.astype(int))
y = df.survived

# then, we create a DataFrame with the attributes
df = df.replace({True: 1, False: 0})
X = df.drop(columns = ['survived'])

# using the scikit-learn to split 30% to test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
X.isnull().sum()

In [ ]:
X.dtypes

In [ ]:
y_train.isnull().sum()

In [ ]:
y.dtypes

### Classification
Classification is a _supervised learning_ method for assigning a label to a sample based on attributes. _Supervised learning_ implies that we have labels for classification or numbers for regression, which the algorithm must learn.<br><br>
In __sklearn__, we will create a model instance and call the _.fit()_ method with the training data and labels. We can then call the _.predict_ method (or the _.predict_proba_ or _.predict_log_proba_ methods) with the model after the adjustments. Finally, to evaluate the model, we can use the _.score()_ method.<br><br>
In general, the biggest challenge is organizing the data to use the __sklearn models__. The data (X) must be in a numpy array (m by n) format or in the pandas DataFrame format. The target data (y) is in a pandas vector or series.
For information, there are several metrics to evaluate how well the model performs on new data.<br><br>
The generic methods that __sklearn models__ implement are:<br>

_fit(X, y[, sample_weight])_
- Adjusts a model

_predict(X)_
- Predict classes

_predict_log_proba(X)_
- Predicts the probability lizarithm

_predict_proba(X)_
- Makes probability prediction

_weight(X, y[, sample_weight])_
- Get accuracy

### Logistic Regression
Logistic Regression estimates probabilities using a logistic function (used for classification).

Efficiency in execution
- You can use *n_jobs* if you are not using the _liblinear_ solver

Interpretation of Results
- The *.coef_* attribute of the model after fitting shows the coefficients of the decision function. A change of x by one unit modifies the log odds ratio according to the coefficient. The *.intercept_* attribute is the inverse of the log odds of the base condition.

In [ ]:
# applying the model
lr = LogisticRegression(random_state = 42)
lr.fit(X_train, y_train)

In [ ]:
# testing the model performance
lr.score(X_test, y_test)

In [ ]:
lr.predict(X.iloc[[0]])

In [ ]:
lr.predict_proba(X.iloc[[0]])

In [ ]:
lr.predict_log_proba(X.iloc[[0]])

In [ ]:
lr.decision_function(X.iloc[[0]])

#### Attributes after fitting
*coef_*
- Decision function coefficients

*intercept_*
- Intercept of the decision function

*m_iter_*
- Number of iterations

#### The intercept is the log odds of the base condition. We can convert it back to a percentage accuracy (probability)

In [ ]:
lr.intercept_

In [ ]:
# creating the logit inverse function
def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

In [ ]:
inv_logit(lr.intercept_)

#### You can inspect the coefficients. The inverse ligit of the coefficients gives us the proportion of positive cases. In other words, for each of the attributes, we can increase or reduce the chance of survival (in the case of the Titanic).

In [ ]:
# defining de the columns to be analysed
cols = X.columns

for col, val in sorted(
    zip(cols, lr.coef_[0]),
    key = lambda x: x[1],
    reverse = True,
):
    print(
        f"{col:10}{val:10.3f} {inv_logit(val):10.3f}"
    )

#### We can use yellowbrick lib to visualize

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))

fi_viz = FeatureImportances(lr)

fi_viz.fit(X, y)

### Naive Bayes
Naive Bayes is a probabilistic classifier that assumes independence between data attributes. It is popular for text classification applications, such as *spam* identification.<br><br>
An advantage of this model is that, by assuming independence between attributes, it is capable of training a model with a small number of samples. A disadvantage is that the model will not be able to capture interactions between attributes.<br><br>
__GaussianNB__
- Assumes a normal distribution

__MultinomialNB__
- Used for discrete occurrence counters

__BernoulliNB__
- Used for discrete Boolean attributes

__Model properties__

_Pre processing_
- it is assumed that the data are independent. Performance should be better if we remove collinear columns. For continuous numeric data, it may be better to separate into bins.

_To avoid overfitting_
- exhibits high bias and low variance (ensembles will not reduce variance).

_Interpretation of results_
- percentage is the probability of a sample belonging to a class based on priors (prior knowledge)

In [ ]:
nb = GaussianNB()

nb.fit(X_train, y_train)

In [ ]:
nb.score(X_test, y_test)

In [ ]:
nb.predict(X.iloc[[0]])

In [ ]:
nb.predict_proba(X.iloc[[0]])

In [ ]:
nb.predict_log_proba(X.iloc[[0]])

__Instance Parameters__

_priors_
- prior probabilities (prior) of classes

_var_smoothing = 1e-9_
- Adding to variance for stable calculations

<br><br>
__Attributes after adaptation__

_class_prior_
- Class probabilities

_class_count_
- Class counters

_theta_
- Average and each column per class

_sigma_
- Variance of each column by class

_epsilon_
- Value to be added for each variance

### Support Vector Machine (SVM)
An SVM is an algorithm that tries to adapt a line between different classes in order to maximize the distance from the line to the class points.<br><br>
__Model properties__<br>
_Efficiency in execution_
- The scikit-learn implementation is O(n4), so it may be difficult to scale to larger sizes. Using a linear kernel or the _LinearSVC_ model can improve execution performance, perhaps at the expense of accuracy.

_Data pre-processing_
- The algorithm is not scale invariant. Standardizing data is highly recommended.

_To avoid overfitting_
- Parameter C (penalty parameter) controls regularization. A smaller value allows for a smaller margin in the hyperplane. A larger value for _gamma_ will tend to overfit the training data.

_Interpretation of results_
- We should inspect .support_vectors_, although they can be difficult to explain. With linear kernels you will be able to inspect the .coef_

In [ ]:
svc = SVC(random_state = 42, probability = True)

svc.fit(X_train, y_train)

In [ ]:
svc.score(X_test, y_test)

In [ ]:
svc.predict(X.iloc[[0]])

In [ ]:
svc.predict_proba(X.iloc[[0]])

In [ ]:
svc.predict_log_proba(X.iloc[[0]])

We use _probability_ = True to obtain the probabilities.

__Instance Parameters__

_C=1.0_
- penalty parameter. The lower the value, the narrower the decision boundary (more overfit).

*cache_size*
- Cache size (MB). Increasing this value can improve training time on large data sets.

*class_weight*
- Dictionary or _balanced_

*coef*
- Independent term for polynomial and sigmoid kernels

*degree*
- Degree for polynomial kernel

*max_iter*
- maximum number of iterations for the solver. -1 indicates there are no limits

*probability*
- False activates probability estimation. Makes training slower

*tol*
- Stop tolerance

*support_*
- Support vector indices

*support_vectors_*
- Support vectors

*n_support_vectors_*
- Number of support vectors per class

*coef_*
- Coefficients for kernel

### K vizinhos mais próximos (KNN)
The KNN (K-Nearest Neighbor) algorithm performs the classification based on the distance to some training samples (k). The family of algorithms is called _instance-based learning_, as there are no parameters to learn. The model assumes that the distance is sufficient to make the inference.<br><br>
The tricky part is selecting the appropriate value of _K_. Furthermore, the curse of dimensionality can hinder distance metrics, as there will be little difference between the closest and most distant neighbors in the case of more dimensions.<br><br>
__Model properties__<br>
_Data pre-processing_
- Distance-based calculations perform better if there is standardization

_To avoid overfitting_
- Elevate *n_neighbors*. Change *p* to L1 or L2 metric

*Interpretation of results*
- Interprets the k nearest neighbors for the sample (using the .kneighbors method)

In [ ]:
knc = KNeighborsClassifier()

knc.fit(X_train, y_train)

In [ ]:
knc.score(X_test, y_test)

In [ ]:
knc.predict(X.iloc[[0]])

In [ ]:
knc.predict_proba(X.iloc[[0]])

__Attributes__<br>
*leaf size*
- Used for tree-based algorithms

*metric*
- Distance metric

*metric_params*
- Additional parameter dictionary for custom metrics function

*n_jobs*
- Number of CPUs

*n_neighbors*
- Number of neighbors

*weights*
- Because it gives *distance*, in which case closer points will have more influence

### Decision Tree
We can use a question and answer process to predict a target class. The advantages of this model include support for non-numeric features, little need for data preparation, support for dealing with non-linear relationships, and finally, we can obtain the importance of the attributes at the end of the model.<br><br>
__Model properties__<br>
*Efficiency in execution*
- goes through each of the attributes and sorts all *n* samples;

*Data pre-processing*
- it is not necessary to climb. You need to get rid of missing values ​​and convert them into numerical data;

*To avoid overfitting*
- set *max_depth* to a smaller number and increase *min_impurity_decrease*

*Interpretation of results*
- because there are steps, a tree is bad at dealing with linear relationships. The tree is also extremely dependent on training data.

In [ ]:
dt = DecisionTreeClassifier(
    random_state=42, max_depth=3
)

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
dt.predict(X.iloc[[0]])

In [ ]:
dt.predict_proba(X.iloc[[0]])

In [ ]:
dt.predict_log_proba(X.iloc[[0]])

__Attributes__<br>
*class_weight*
- class weights in a dictionary. *balanced* defines values in inverse proportion to class frequencies

*criterion='gini'*
- separation function, *gini* or *entropy*

*max_depth=None*
- depth of the tree

*max_features=None*
- number of attributes to be analyzed for separation. The default value is all

*max_leaf_nodes=None*
- limitates the number of leaves

*min_impurity_decrease=0.0*
- separates a node if the separation decreases the impurity by an amount greater than or equal to the defined

*min_samples_leaf*
- minimum number of samples required to separate a node<br><br>


**Attributes after adjustment**

*classes_*
- class label

*feature_importances_*
- array with the importance of each feature

*n_classes_*
- number of classes

*n_features_*
- number of attributes

#### Viewing the implemented tree

In [ ]:
'''
dot_data = StringIO()

tree.export_graphviz(
    dt,
    out_file=dot_data'
    feature_names=["Died", "Survived"],
    filled=True
)

g = pydotplus.graph_from_dot_data(
    dot_data.getvalue()
)
'''

In [ ]:
'''
import pydotplus
from io import StringIO
from sklearn.tree import export_graphviz
dot_data = StringIO()
tree.export_graphviz(
    dt,
    out_file=dot_data,
    feature_names=X.columns,
    class_names=["Died", "Survived"],
    filled=True'
)
g = pydotplus.graph_from_dot_data(
    dot_data.getvalue()
)

g.write("py-projectsdata-science-projectsmachine-learning-pocket-reference-tree_classifier_view.png")
'''

In [ ]:
# Image(g.create_png())

#### Visualizing DecisionTree, which provides valuable visual insights

In [ ]:
# using tree.plot_tree to visualize DecisionTreeClassifier
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt
                  )

In [ ]:
# using plot_tree to view DecisionTreeClassifier
plot_tree(dt)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

plot_tree(dt)

fig.savefig("images/classification_decision_tree.png")

In [ ]:
# using dtreeviz to view DecisionTreeClassifier
viz = dtreeviz.model(dt, X, y,
               target_name="survived",
               feature_names=X.columns,
               class_names=["died", "survived"]
)

viz.view()

In [ ]:
# using text_representation to view DecisionTreeClassifier
text_representation = tree.export_text(dt)
print(text_representation)

#### Feature Importances

In [ ]:
for col, val in sorted(
    zip(X.columns, dt.feature_importances_),
    key=lambda x: x[1],
    reverse=True
)[:5]:
    print(f"{col:10}{val:10.3f}")

#### We can use Yellowbrick to visualize the Feature Importances

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

fi_viz = FeatureImportances(dt)

fi_viz.fit(X, y)

### Random Forest
A Random Forest is a set of decision trees. She uses *bagging* to correct the trees' tendency to overfitting. Random Forest provides us with feature importances by calculating the average importance across all trees.<br><br>
**Model Properties**<br>
*Efficiency in execution*
- Create *j* using *n_jobs*. The complexity of each tree is O(mn log n(, where *n* is the number of samples and *m* the number of attributes.

*Data pre-processing*
- it is not necessary

*To avoid overfitting*
- add more trees (*n_estimators*). Use a smaller value for *max_depth*

*Interpretation of results*
- has support for feature importances

In [ ]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
rf.predict(X.iloc[[0]])

In [ ]:
rf.predict_proba(X.iloc[[0]])

In [ ]:
rf.predict_log_proba(X.iloc[[0]])

**Instance Parameters**

*class_weight*
- class weights in a dictionary. *balanced* defines values in inverse proportion to class frequencies

*criterion='gini'*
- separation function, *gini* or *entropy*

*max_depth=None*
- depth of the tree

*max_features=None*
- number of attributes to be analyzed for separation. The default value is all

*max_leaf_nodes=None*
- limitates the number of leaves

*min_impurity_decrease=0.0*
- separates a node if the separation decreases the impurity by an amount greater than or equal to the defined

*min_samples_leaf*
- minimum number of samples required to separate a node

*n_estimators*
- number of trees in the forest

*n_jobs*
- number of jobs to be carried out for adaptation and prediction<br><br>


**Attributes after Adjustment**

*classes_*
- class label

*feature_importances_*
- array with the importance of each feature

*n_classes_*
- number of classes

*n_features_*
- number of attributes

#### Feature Importances

In [ ]:
for col, val in sorted(
    zip(X.columns, rf.feature_importances_),
    key=lambda x: x[1],
    reverse=True
)[:6]:
    print(f"{col:10}{val:10.3f}")

The random forest classifier calculates the importance of attributes by determining the *average impurity decrease* for each attribute. Attributes that reduce classification uncertainty receive higher scores.<br><br>
These numbers may become inaccurate if the attributes vary in scale or cardinality of the category columns. A more reliable score is the *importance of the permutation*. A more reliable method is *discarded column importance*.

In [ ]:
rf2 = RandomForestClassifier(random_state=42)

rf2.fit(X_train, y_train)

rfpimp.importances(
    rf2, X_test, y_test
).Importance

### XGBoost

XGBoost is a popular library. It creates a weak tree and then "improves" subsequent trees (boosting) in order to reduce residual errors. The algorithm attempts to capture and handle any patterns in the errors until they appear to be random.<br><br>
**Model Properties**<br>
*Efficiency in execution*
- XGBoost can run in parallel. Using the *n_jobs* function to inform the number of CPUs.

*Data pre-processing*
- no need to scale in tree-based models. Category data must be encoded

*To avoid overfitting*
- the parameter *early_stopping_rounds=N* can be defined to stop training if there are no improvements after N rounds.

*Interpretation of results*
- it is possible to include the importance of attributes

In [ ]:
# early_stopping_rounds shoub be passed on XGBClassifier()
xgb_class = xgb.XGBClassifier(random_state=42, early_stopping_rounds=10)

xgb_class.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)]
)

In [ ]:
xgb_class.predict(X.iloc[[0]])

In [ ]:
xgb_class.predict_proba(X.iloc[[0]])

**Instance parameters**<br>
*max_depth*
- maximum depth

*learning_rate*
- learning rate for boosting. After each execution step, the newly added weights are scaled according to this factor. The lower the value, the more conservative it will be, but more trees will be needed to converge. In the *.train* call, we can pass a *learning_rates* parameter, which is a list of rates in each round

*n_estimators*
- number of rounds or improved trees

*n-jobs*
- number of threads to be used

*gamma*
- controls pruning. Ranges from 0 to infinity. Minimum loss reduction required to separate one more sheet. The higher the gamma value, the more conservative it will be.

*subsample*
- fraction of samples to be used in the next round
<br><br>

**Attributes** <br>
*coef_*
- coefficients for gblinear learners

*feature_importances_*
- importance of attributes. The importance of attributes is the average gain across all nodes where the attribute is used

In [ ]:
for col, val in sorted(
    zip(
        X.columns,
        xgb_class.feature_importances_,
    ),
    key=lambda X: X[1],
    reverse=True
)[:6]:
    print(f"{col:10}{val:10.3f}")

#### Generating a graph of the importance of attributes with XGBoost
XGBoost has *importance_type* parameter. The default value is *weight*, which is the number of times. an attribute appears in a tree. It can be *gain*, which would be the average gain when the attribute is used, and it can also be *cover*, which would be the number of samples affected by a separation.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

xgb.plot_importance(xgb_class, ax=ax)

fig.savefig("images/classification_xgboost_feature_importances.png")

#### Generating a graph of the importance of attributes with Yellowbrick

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

fi_viz = FeatureImportances(xgb_class)

fi_viz.fit(X, y)

fig.savefig("images/classification_yellowbrick_xgboost_feature_importances")

#### XGBoost offers a textual representation

In [ ]:
booster = xgb_class.get_booster()

print(booster.get_dump()[0])

The value on the sheet is the score for class 1. It can be converted to a probability using the logistic function.

Next, we will plot the graphical version of the first model tree.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

xgb.plot_tree(xgb_class, ax=ax, num_trees=0)

fig.savefig("images/classification_first_tree_xgboost")

#### xgbfir package
The xgb fit package is a lib developed based on XGBoost. This lib provides several measures related to the importance of attributes. Its unique feature is that it provides these measurements over columns, and over pairs of columns as well, so that you can see the interactions. Furthermore, we can obtain information about interactions between column cracks.

**Measurements**<br><br>
*Gain*
- total gain of each attribute or interaction between attributes

*FScore*
- number of possible separations in an attribute or interaction between attributes

*wFScore*
- number of possible separations in an attribute or interaction between attributes, with weights assigned according to the probability of the separations occurring

*Average wFScore*
- wFScore divided by FScore

*Average Gain*
- gin divided by FScore

*Expected Gain*
- total gain for each attribute or interaction between attributes, with weights according to the probability of obtaining the gain

In [ ]:
xgbfir.saveXgbFI(
    xgb_class,
    feature_names=X.columns,
    OutputXlsxFile="temp/measurements_xgb_fir.xlsx",
)

# selecting 3 main attributes
pd.read_excel("temp/measurements_xgb_fir.xlsx").head(3).T

From this table, we can see that *sex_female* has a high position in terms of **Gain**, **Average wDScore**, **Average gain** and **Expected gain**, while *sib_sp* is highlights in **FScore** and **wFScore**<br><br>
Let's analyze the pairs to see the interactions between the columns

In [ ]:
pd.read_excel(
       "temp/measurements_xgb_fir.xlsx",
        sheet_name='Interaction Depth 1',
).head(5).T

In this case, we can see that the two main interactions involve the *sex_female* and *pclass* column. If we had to choose just 2 attributes for our model, we would probably choose *sex_female* and *pclass*<br><br>
Next, we will analyze the column triplets

In [ ]:
pd.read_excel(
       "temp/measurements_xgb_fir.xlsx",
        sheet_name='Interaction Depth 2',
).head(5).T

### LightGBM
LightGBM is a Microsoft implementation. It uses a sampling method to deal with continuous values. This allows for faster tree creation (compared to XGBoost), as well as reducing memory usage.<br>
LightGBM also creates trees in depth by leaves, rather than levels. Because of this, instead of using *max_depth* to control overfitting, we use *num_leaves*.<br><br>
**Model properties**<br><br>
*Efficiency in execution*
- can take advantage of multiple CPUs

*Data pre-processing*
- has some support for encoding category columns as integers

*To avoid overfitting*
- reduce *num_leaves*, increase *min_data_in_leaf* and use *min_gain_to_split* with **lambda_L1** and **lambda_L2**

*Interpretation of results*
- the importance of attributes is available

In [8]:
lgbm_class = lgb.LGBMClassifier(
    random_state=42
)

In [9]:
lgbm_class.fit(X_train, y_train)

NameError: name 'X_train' is not defined